## Data setup

In [4]:
%%bash
python ./generate_data.py

In [5]:
from pyspark.sql import SparkSession

# Create Spark session
spark = SparkSession.builder \
    .appName("Joins") \
    .getOrCreate()

25/06/14 09:24:40 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [6]:


# Drop existing tables if they exist
spark.sql("DROP TABLE IF EXISTS prod.db.customer")
spark.sql("DROP TABLE IF EXISTS prod.db.lineitem")
spark.sql("DROP TABLE IF EXISTS prod.db.nation")
spark.sql("DROP TABLE IF EXISTS prod.db.orders")
spark.sql("DROP TABLE IF EXISTS prod.db.part")
spark.sql("DROP TABLE IF EXISTS prod.db.partsupp")
spark.sql("DROP TABLE IF EXISTS prod.db.region")
spark.sql("DROP TABLE IF EXISTS prod.db.supplier")



DataFrame[]

In [7]:
# Create tables using Iceberg format
spark.sql("""
CREATE TABLE IF NOT EXISTS prod.db.customer (
  c_custkey    BIGINT,
  c_name       STRING,
  c_address    STRING,
  c_nationkey  BIGINT,
  c_phone      STRING,
  c_acctbal    DECIMAL(15,2),
  c_mktsegment STRING,
  c_comment    STRING
) USING iceberg
TBLPROPERTIES (
  'format-version' = '2'
)
""")

spark.sql("""
CREATE TABLE IF NOT EXISTS prod.db.lineitem (
  l_orderkey      BIGINT,
  l_partkey       BIGINT,
  l_suppkey       BIGINT,
  l_linenumber    INT,
  l_quantity      DECIMAL(15,2),
  l_extendedprice DECIMAL(15,2),
  l_discount      DECIMAL(15,2),
  l_tax           DECIMAL(15,2),
  l_returnflag    STRING,
  l_linestatus    STRING,
  l_shipdate      DATE,
  l_commitdate    DATE,
  l_receiptdate   DATE,
  l_shipinstruct  STRING,
  l_shipmode      STRING,
  l_comment       STRING
) USING iceberg
TBLPROPERTIES (
  'format-version' = '2'
)
""")

spark.sql("""
CREATE TABLE IF NOT EXISTS prod.db.nation (
  n_nationkey INT,
  n_name      STRING,
  n_regionkey INT,
  n_comment   STRING
) USING iceberg
TBLPROPERTIES (
  'format-version' = '2'
)
""")

spark.sql("""
CREATE TABLE IF NOT EXISTS prod.db.orders (
  o_orderkey      BIGINT,
  o_custkey       BIGINT,
  o_orderstatus   STRING,
  o_totalprice    DECIMAL(15,2),
  o_orderdate     DATE,
  o_orderpriority STRING,
  o_clerk         STRING,
  o_shippriority  INT,
  o_comment       STRING
) USING iceberg
TBLPROPERTIES (
  'format-version' = '2'
)
""")

spark.sql("""
CREATE TABLE IF NOT EXISTS prod.db.part (
  p_partkey     BIGINT,
  p_name        STRING,
  p_mfgr        STRING,
  p_brand       STRING,
  p_type        STRING,
  p_size        INT,
  p_container   STRING,
  p_retailprice DECIMAL(15,2),
  p_comment     STRING
) USING iceberg
TBLPROPERTIES (
  'format-version' = '2'
)
""")

spark.sql("""
CREATE TABLE IF NOT EXISTS prod.db.partsupp (
  ps_partkey    BIGINT,
  ps_suppkey    BIGINT,
  ps_availqty   INT,
  ps_supplycost DECIMAL(15,2),
  ps_comment    STRING
) USING iceberg
TBLPROPERTIES (
  'format-version' = '2'
)
""")

spark.sql("""
CREATE TABLE IF NOT EXISTS prod.db.region (
  r_regionkey INT,
  r_name      STRING,
  r_comment   STRING
) USING iceberg
TBLPROPERTIES (
  'format-version' = '2'
)
""")

spark.sql("""
CREATE TABLE IF NOT EXISTS prod.db.supplier (
  s_suppkey   BIGINT,
  s_name      STRING,
  s_address   STRING,
  s_nationkey BIGINT,
  s_phone     STRING,
  s_acctbal   DECIMAL(15,2),
  s_comment   STRING
) USING iceberg
TBLPROPERTIES (
  'format-version' = '2'
)
""")

DataFrame[]

In [11]:
from pathlib import Path
def upsert_data(data_name, data_path = Path('/home/iceberg/notebooks/notebooks/data')):
    csv_path = data_path / f'{data_name}.csv'
    print(f'Reading {data_name} data from {str(csv_path)}')
    df = spark.read.format("csv").option("header", "true").option("delimiter", ",").option("inferSchema", "true").load(str(csv_path))
    df.writeTo(f"prod.db.{data_name}").overwritePartitions()
    

In [12]:
upsert_data('customer')
upsert_data('lineitem')
upsert_data("nation")
upsert_data("orders")
upsert_data("part")
upsert_data("partsupp")
upsert_data("region")
upsert_data("supplier")


Reading customer data from /home/iceberg/notebooks/notebooks/data/customer.csv


Reading lineitem data from /home/iceberg/notebooks/notebooks/data/lineitem.csv


Reading nation data from /home/iceberg/notebooks/notebooks/data/nation.csv
Reading orders data from /home/iceberg/notebooks/notebooks/data/orders.csv


Reading part data from /home/iceberg/notebooks/notebooks/data/part.csv
Reading partsupp data from /home/iceberg/notebooks/notebooks/data/partsupp.csv


Reading region data from /home/iceberg/notebooks/notebooks/data/region.csv
Reading supplier data from /home/iceberg/notebooks/notebooks/data/supplier.csv


In [15]:
%%sql
select * from prod.db.customer limit 5

c_custkey,c_name,c_address,c_nationkey,c_phone,c_acctbal,c_mktsegment,c_comment
1,Customer#000000001,j5JsirBM9PsCy0O1m,15,25-989-741-2988,711.56,BUILDING,y final requests wake slyly quickly special accounts. blithely
2,Customer#000000002,487LW1dovn6Q4dMVymKwwLE9OKf3QG,13,23-768-687-3665,121.65,AUTOMOBILE,y carefully regular foxes. slyly regular requests about the bli
3,Customer#000000003,fkRGN8nY4pkE,1,11-719-748-3364,7498.12,AUTOMOBILE,fully. carefully silent instructions sleep alongside of the slyly regular asymptotes. quickly regular
4,Customer#000000004,4u58h fqkyE,4,14-128-190-5944,2866.83,MACHINERY,sublate. fluffily even instructions are about th
5,Customer#000000005,hwBtxkoBF qSW4KrIk5U 2B1AU7H,3,13-750-942-6364,794.47,HOUSEHOLD,equests haggle furiously against the pending packa


## Facts & Dimensions

1. `Fact` tables containing information about how dimensions interact with each other in real life. Example: An order fact is an interaction between a customer and a seller involving one or more products.
2. `Dimension` tables store data for a business entity (e.g., customer, product, partner, etc). These tables describe the ‘who’ and ‘what’ types of questions. For example, which stores had the highest revenue yesterday? In this question, stores will be the dimension.

Analytical query

## Joins are essential for creating dimension tables & reporting

### Creating dimension tables from normalized upstream tables

- Multiple normalized tables from upstream are combines to form dimension table
- Joins are also used to enrich fact tables with dimensional information

### Types of join & when to use them

Join criteria refers to the columns used to join the tables. When joining tables, there is usually one table called the driver table to which other tables are joined.

Depending on your use case, you may want to:

1. Use a left join to get all data from the driver table, even when relevant data is missing from other tables.
2. Use an inner join to retrieve only data that is present in all the tables in the join.
3. Use an anti-join to retrieve data from the driver table that is not present in the table being joined to.
4. Use a full outer join to get data in either one of the multiple joining tables. This type of join is typically used for data validation and to determine differences in data between the tables.


### Joins lead to bad data if underlying data assumptions are incorrect

1. Joining table(s) with multiple grains will lead to duplicated or partial data. It’s best to ensure that your table(s) have a single grain before joining them.
2. If the tables you are joining do not have complete data, your joins will produce partial or NULL data points. For e.g., if you are joining a customer’s personal details table with their payment information and the payment information for that customer has not yet arrived in the warehouse, you will receive NULL for this information.
3. Joins on columns with NULLs. NULLs represent the absence of data, and as such, joins on NULL = NULL will not work. Ensure that you coalesce NULLs to a hardcoded value before joining (if that’s what you intend to do).
4. Complex join criteria and join criteria that require transformation functions are typically indicative of a poor underlying data model. Try to solve this by making changes to the tables in the join.


## Group by represents the creation of metrics for analytics

### Group by enables humans to understand data quickly


### Go beyond standard aggregate functions.
1. Standard agg: min/max/sum/avg/count
2. Statistical agg: Functions like correlation, sampling, standard deviation, skew, etc
3. Collection agg: Functions to combine values into nested data types, e.g., array_agg, collect_set, etc
4. Approximation agg: Functions that are fast by sacrificing accuracy, e.g., approx_distinct, approx_most_frequent
5. Convenience agg: Functions that make common usages easier, e.g., count_if, bool_or, etc


### Underlying data model and types need to be correct for group by to work as expected

1. If you are grouping by multiple columns or using Group by to remove duplicates, this usually indicates a problem with your underlying data model
2. For the columns on which you want to run aggregation functions, ensure that they are of the correct data type.
3. Ensure that the columns that you are aggregating are additive. For example, you cannot aggregate percentages; instead, you must aggregate the numerator and denominator separately.
